In [66]:
# Comprobamos si está tensorflow-gpu==2.3.0
# !pip freeze

In [67]:
# Hay que instalar esto si se quiere utilizar la gpu
#!pip install tensorflow-gpu==2.3.0 

In [68]:
# Esta es la tarjeta grafica
# !nvidia-smi

---
---

# Instalamos las librerías necesarias

In [69]:
# instalar librerías. Esta casilla es últil por ejemplo si se ejecuta el cuaderno en Google Colab
# Note que existen otras dependencias como tensorflow, etc. que en este caso se encontrarían ya instaladas
%%capture
!pip install transformers==4.2.1
# !pip install tensorflow-gpu==2.3.0
!pip install stop_words
print('Done!')

In [70]:
# instalar librerías. Esta casilla es últil por ejemplo si se ejecuta el cuaderno en Google Colab
# Note que existen otras dependencias como tensorflow, etc. que en este caso se encontrarían ya instaladas

"""%%capture
# Librería transformers
try:
    import tranformers
    print("module 'tranformers' is installed")
except ModuleNotFoundError:
    print("module 'transformers' is being installed")
    !pip install transformers==4.2.1
# Por si se quiere usar la gpu verificamos si tenemos transformers-gpu
import sys
if "tensorflow-gpu" in sys.modules:
    print("tensorflow-gpu already in sys.modules")
else: 
  !pip install tensorflow-gpu==2.3.0"""

'%%capture\n# Librería transformers\ntry:\n    import tranformers\n    print("module \'tranformers\' is installed")\nexcept ModuleNotFoundError:\n    print("module \'transformers\' is being installed")\n    !pip install transformers==4.2.1\n# Por si se quiere usar la gpu verificamos si tenemos transformers-gpu\nimport sys\nif "tensorflow-gpu" in sys.modules:\n    print("tensorflow-gpu already in sys.modules")\nelse: \n  !pip install tensorflow-gpu==2.3.0'

---
---

# Importamos librerías

In [71]:
#%reset
import pandas as pd
import tensorflow as tf
from transformers import DistilBertConfig, DistilBertTokenizer, TFDistilBertModel
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
# Para trabajar teniendo encuenta el desbalanceo
from sklearn.utils import class_weight
import os
import tqdm
import numpy as np
from tensorflow.keras import backend as K

# Librerías para la matriz word context
from nltk.stem.snowball import SpanishStemmer
from sklearn.feature_extraction.text import CountVectorizer
from stop_words import get_stop_words # Se añade librería para obtener las stop_word de cualquier idioma
import pickle

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [72]:
from tensorflow.keras.layers import Dense, Input, BatchNormalization, LeakyReLU, Activation, Dropout, Conv1D, Reshape, Flatten, MaxPooling1D, Concatenate
from tensorflow.keras.activations import softmax, tanh, sigmoid, relu
from tensorflow.keras.models import Model

---
---

# Cargamos los datos

In [73]:
cleaned_corpus = pd.read_csv("https://raw.githubusercontent.com/jibt1/competition_group/main/Tareas/task_preproceso/Super_Final_versionV3/cleaned_corpus.csv", sep=',')
cleaned_corpus = cleaned_corpus[~cleaned_corpus.text_v5.isna()]
print(cleaned_corpus.shape)
cleaned_corpus.head()

(23984, 28)


,id,text,text_v2,text_v3,text_v4,text_v5,en_text,len_token,n_char,ratio,detected_cansinos,detected_emojis_cansinos,detected_text_cansinos,text_emojis,HasNoun,HasAdj,HasVerb,PoSTags,is_humor,votes_no,votes_1,votes_2,votes_3,votes_4,votes_5,humor_rating,humor_mechanism,humor_target
0,tweet1,Niveles de retraso mental: \n\n— Bajo.\n— Medi...,Niveles de retraso mental: \n\n— Bajo.\n— Medi...,Niveles de retraso mental: \n\n— Bajo.\n— Medi...,Niveles de retraso mental . Bajo . . Medio...,Niveles de retraso mental . Bajo . . Medio...,mental retardation levels . low . . medi...,19,86,0.220930,0,0,0,no_emojis,1.0,1.0,0.0,NOUN ADP NOUN ADJ SPACE PUNCT SPACE PROPN PUNC...,1,1,2,2,0,0,0,1.5,NaN,NaN
1,tweet2,"—Vamos Luke desenfunda tu sable, demuestra tu ...","—Vamos Luke desenfunda tu sable, demuestra tu ...","—Vamos Luke desenfunda tu sable, demuestra tu ...",. Vamos Luke desenfunda tu sable demuestra tu...,. Vamos Luke desenfunda tu sable demuestra tu...,. let's luke unscrew your sabre shows your h...,22,140,0.157143,0,0,0,no_emojis,1.0,1.0,1.0,SPACE PUNCT VERB PROPN VERB DET NOUN VERB DET ...,1,1,3,0,1,0,0,1.5,NaN,NaN
2,tweet3,"- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ...","- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ...","- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ...","- ¿ Te ofrezco algo ? , ¿ agua café mi coraz...","- ¿ Te ofrezco algo ? , ¿ agua café mi coraz...","- Do I offer you anything? , do I water coffee...",21,112,0.187500,1,0,1,no_emojis,1.0,1.0,1.0,PUNCT SPACE PUNCT PRON VERB PRON PUNCT PUNCT S...,1,0,2,1,0,1,1,2.6,NaN,NaN
3,tweet4,No se porqué me hago la cabeza deooos,No se porqué me hago la cabeza deooos,No se porqué me hago la cabeza deos,No se porqué me hago la cabeza deos,No se porqué me hago la cabeza deos,I don't know why I'm doing my head.,8,37,0.216216,1,0,1,no_emojis,1.0,1.0,1.0,ADV PRON VERB PRON VERB DET NOUN ADJ,0,3,0,0,0,0,0,NaN,NaN,NaN
4,tweet5,Quisiera saber que hago durante la siesta de l...,Quisiera saber que hago durante la siesta de l...,Quisiera saber que hago durante la siesta de l...,Quisiera saber que hago durante la siesta de l...,Quisiera saber que hago durante la siesta de l...,I want to know what I do during the nap from w...,20,106,0.188679,0,0,0,no_emojis,1.0,1.0,1.0,VERB AUX SCONJ VERB ADP DET NOUN ADP DET PRON ...,0,4,0,1,0,0,0,NaN,NaN,NaN


In [74]:
#test_dataframe = pd.read_csv("https://raw.githubusercontent.com/jibt1/competition_group/main/datasets/haha_2021_dev.csv", sep=',')

---
---

# Métricas para el entrenamiento

In [75]:
!pip install tensorflow_addons
import tensorflow_addons as tfa 

In [76]:
# Se definen metricas

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

---
---

# Establecemos la configuración de DistilBert

In [77]:
# https://huggingface.co/transformers/model_doc/distilbert.html
# https://huggingface.co/dccuchile/bert-base-spanish-wwm-uncased
# dim dimension del pooling layer de los outputs del encoder en la salida de la ultima capa
# dropout es el dropout de las denses de las capas de encoders
# seq_classif_dropout es el dropout de la ultima densa ajena a Bert (top dense)

---
---

# Clase Modelo Fine Tuning

In [78]:
class CustomDistilBERTModel():
    """
    Se crea una clase con todo lo necesario para construir nuestro modelo
    """

    def __init__(self, cleaned_corpus, stop_words):
          """
          Atributos:

          cleaned_corpus (df): corpus tratado a partir del ipynb con el Preprocesado de Datos
                               `PreprocesadoTextos_SuperFinalV.ipynb`

          stop_words (list): lista con las palabras que se quieren excluir del conocimiento específico
          """

          #### Configuración del modelo ####

          # cfg (dict): diccionario con los datos generales que configuran el modelo DistilBert
          self.cfg = {}
          self.cfg["framework"] = "tf" # Tensorflow
          self.cfg["max_length"] = 256
          # Modelo Bert para Spanish
          self.cfg["transformer_model_name"] = "dccuchile/bert-base-spanish-wwm-cased"
          # El problema es binario pero en lugar de una neurona utilizo 2 neuronas
          self.cfg["num_labels"] = 2

          # Cargamos el tokenizador correspondiente ¿Lematiza?
          self.cfg['tokenizer'] = DistilBertTokenizer.from_pretrained(self.cfg['transformer_model_name'],
                                                                      output_attentions=True, return_dict=True)
          # Proceso de scikit learn para hacer OHE a 0 1 de la salida
          self.cfg['label_binarizer'] = preprocessing.LabelBinarizer()

          # config_bert (dict): diccionario con los datos que filtran y disminuyen las capas de DistilBert
          self.config_bert = DistilBertConfig(num_labels = self.cfg["num_labels"], 
                              attention_dropout=0.75, seq_classif_dropout=0, dropout=0.75,
                              n_heads=4, dim=64, max_position_embeddings=self.cfg["max_length"],
                              n_layers=2, hidden_dim=128, vocab_size=self.cfg['tokenizer'].vocab_size,
                              output_hidden_states=True)
          
          # Instanciamos DistilBertModel
          self.DistilBert = TFDistilBertModel.from_pretrained(self.cfg["transformer_model_name"],
                                                      config=self.config_bert)
          # Futuro modelo keras
          self.humor_model = None

          # Corpus
          self.corpus = cleaned_corpus

          # Stop Words
          self.stop_words = stop_words

          # Count Vectorizer
          self.c_vec = CountVectorizer(analyzer=self.spanish_stemmer, stop_words=self.stop_words, lowercase=True)

    def spanish_stemmer(self, sentence):
        """
        Método para obtener el lema de cada token del corpus

        Args:
          sentence: tweet en formato string
        """

        stemmer = SpanishStemmer()
        analyzer = CountVectorizer(binary=False, analyzer='word', stop_words=self.stop_words,
                                  ngram_range=(1, 1)).build_analyzer()
        return (stemmer.stem(word) for word in analyzer(sentence))
        
    def get_word_context_matrix(self):
        """
        Método que obtiene la matriz word_context para un corpus dado

        Args:
          which_text: cualesquiera de los pd.Series del corpus con acceso:
                  text, text_v1, text_v2, text_v3, text_v4, text_v5
        """
        self.word_context_matrix = self.c_vec.fit_transform(self.x_train[self.which_text].to_list())
        # self.save_object(self.count_vectorizer, "pickle_count_vectorizer")
        #return word_context_matrix

    def get_model_inputs(self, is_val=False):
        """
        Método que obtiene los inputs_ids (mapeo del vocabulario) y las máscaras
        de atención. Ambos son inputs necesarios para el modelo de Bert
        """
        if is_val==False:
          corpus_list = self.x_train[self.which_text].to_list()
        else:
          corpus_list = self.x_val[self.which_text].to_list()
        
        encodings = self.cfg['tokenizer'](corpus_list, truncation=True, padding='max_length',
                                          max_length=self.cfg['max_length'], return_tensors=self.cfg['framework'])
        inputs = {'input_ids': encodings['input_ids'],
                'attention_mask': encodings['attention_mask']
                }
        return inputs

    def prepare_inputs(self, which_text, which_target):
        """
        Método que prepara los inputs de entrada
        """
        self.which_text = which_text
        self.which_target = which_target
        self.get_word_context_matrix()
        self.get_model_inputs()

    def DistilBert_extension(self):#, ids, mask):
        """
        Método que crea la arquitectura y su correspondiente flujo y que, además,
        compila dicha arquitectura
        """

        input_ids = Input(shape=(256, ),dtype='int32')
        attention_mask = Input(shape=(256, ), dtype='int32')
        word_context = Input(shape=(self.word_context_matrix.shape[1],), dtype='int32')

        # https://huggingface.co/transformers/model_doc/distilbert.html#distilbertmodel

        output_bert = self.DistilBert(
                                  input_ids=input_ids, 
                                  attention_mask=attention_mask
                                  )

        """
        last_hidden_state, 1 tensor
        hidden_states, 3 tensores (n_layers = 2 + entrada)
        attention = None (en principio)
        """
        # output_bert = output_bert['hidden_states']
        output_bert = output_bert['last_hidden_state'][:,0,:]
        """
        (batch_size, longitud_secuencia, profundidad de las capas FF intermedias)
        (batch_size, max_position_embeddings, dim=32)
        """

        # output_bert = Concatenate()(output_bert)

        # flatten_bert = Flatten()(output_bert)

        dense_word_context = Dense(128)(word_context)

        dense = Concatenate()([output_bert, dense_word_context])

        dense = Dense(64)(dense)
        dense = BatchNormalization()(dense)
        dense = LeakyReLU()(dense)
        dense = Dropout(0.4)(dense)
        dense = Dense(32)(dense)
        dense = BatchNormalization()(dense)
        dense = relu(dense)
        dense = Dropout(0.2)(dense)

        output = Dense(2, activation='softmax')(dense)

        ### Output
        model = Model([input_ids, attention_mask, word_context], output)

        ### Cross Entropy y Optimizador Adam
        optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
        loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True)

        # Compilación del modelo
        model.compile(optimizer=optimizer, loss=loss, metrics=[tfa.metrics.F1Score(2, 'macro'),  'accuracy'])
        model.summary()

        self.humor_model = model

    def train_model(self, which_text='text_v5', which_target='is_humor', pval=0):
        """
        Método que entrena el modelo.

        Args:

        * input_ids:
        * attention_mask:
        * word_context
        """

        if pval < 0 or pval >= 1:
          raise ValueError('0 <= pval < 1 es la proporción del conjunto de validación')

        if pval > 0:
          X = self.corpus
          y = self.corpus[[which_target]]
          x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=pval, stratify=y)
          self.entire_dataset_for_training = False

          self.x_train = x_train
          self.y_train = y_train
          self.x_val = x_val
          self.y_val = y_val

          # Preparamos los datos
          self.prepare_inputs(which_text, which_target)

          self.val_word_context = self.c_vec.transform(x_val[self.which_text])

          train_blabels = to_categorical(self.y_train)
          train_blabels_t = tf.convert_to_tensor(train_blabels, dtype='int32')
          val_blabels = to_categorical(y_val)
          val_blabels_t = tf.convert_to_tensor(val_blabels, dtype='int32')

          train_inputs = self.get_model_inputs()
          val_inputs = self.get_model_inputs(is_val=True)

          earlyStopping = EarlyStopping(monitor='val_accuracy', patience=8, verbose=0, mode='max')
          checkpoint = ModelCheckpoint("best_model", monitor="val_accuracy", save_best_only=True)

          class_weights = class_weight.compute_class_weight('balanced', np.unique(train_blabels), train_blabels.ravel())
          class_weights = dict(enumerate(class_weights))

          # Compilamos
          self.DistilBert_extension()

          self.humor_model.fit([train_inputs['input_ids'], train_inputs['attention_mask'], 
                      self.word_context_matrix.toarray()],  y=train_blabels_t,
                      validation_data=([val_inputs['input_ids'], val_inputs['attention_mask'], self.val_word_context.toarray()], val_blabels_t),
                      batch_size = 32, epochs = 10,
                      verbose = 1, callbacks=[earlyStopping, checkpoint], class_weight=class_weights)
        else:
          self.x_train = self.corpus
          self.y_train = self.corpus[which_target]
          train_blabels = to_categorical(self.y_train)
          train_blabels_t = tf.convert_to_tensor(train_blabels, dtype='int32')

          # Preparamos los datos
          self.prepare_inputs(which_text, which_target)

          train_inputs = self.get_model_inputs()

          class_weights = class_weight.compute_class_weight('balanced', np.unique(train_blabels), train_blabels.ravel())
          class_weights = dict(enumerate(class_weights))

          # Compilamos
          self.DistilBert_extension()
          checkpoint = ModelCheckpoint("best_model", monitor="val_accuracy", save_best_only=True)
          self.humor_model.fit([train_inputs['input_ids'], train_inputs['attention_mask'], 
                      self.word_context_matrix.toarray()],  y=train_blabels_t,
                      batch_size = 32, epochs = 10,
                      verbose = 1, callbacks=[checkpoint], class_weight=class_weights)

        # Empleamos el labelizador
        #cfg['label_binarizer'].fit(train["is_humor"], num_labels=2)


        # Instancio el méétodo earlyStopping para no perseverar en el entrenamiento si
        # se obtiene una mejora tras una paciencia de 8 iteraciones
        earlyStopping = EarlyStopping(monitor='val_accuracy', patience=8, verbose=0, mode='max')

        # Creo un guardado del mejor modelo visto hasta el fin del entrenamiento monitorizando
        # el val_accuracy
        # checkpoint = ModelCheckpoint("best_model.h5", monitor="val_accuracy", save_best_only=True)

In [79]:
!wget "https://raw.githubusercontent.com/jibt1/competition_group/main/Tareas/datasets/more_stop_words.txt"

# Se obtienen las stop_words en español

with open("more_stop_words.txt", "rb") as f:
  list_test = pickle.load(f)

stop_words = get_stop_words("spanish")
more_stop_words = stop_words + list_test
more_stop_words.sort()

--2021-05-24 14:33:20--  https://raw.githubusercontent.com/jibt1/competition_group/main/Tareas/datasets/more_stop_words.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10258 (10K) [application/octet-stream]
Saving to: ‘more_stop_words.txt.4’

more_stop_words.txt 100%[===================>]  10.02K  --.-KB/s    in 0s      

2021-05-24 14:33:20 (76.7 MB/s) - ‘more_stop_words.txt.4’ saved [10258/10258]



In [80]:
# Instanciamos el modelo
instancia = CustomDistilBERTModel(cleaned_corpus, more_stop_words)

Some layers from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing TFDistilBertModel: ['mlm___cls', 'bert']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['distilbert']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [81]:
instancia.train_model(pval=0.15)

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           [(None, 256)]        0                                            
__________________________________________________________________________________________________
tf_distil_bert_model_4 (TFDisti TFBaseModelOutput(la 2067584     input_13[0][0]                   
                                                                 input_14[0][0]                   
__________________________________________________________________________________________________
input_15 (InputLayer)           [(None, 16180)]      0                                      

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Epoch 1/10


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


...

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
T

INFO:tensorflow:Assets written to: best_model/assets


INFO:tensorflow:Assets written to: best_model/assets


Epoch 10/10
638/638 [==============================] - 20s 31ms/step - loss: 0.3002 - f1_score: 0.8689 - accuracy: 0.8776 - val_loss: 0.3758 - val_f1_score: 0.8420 - val_accuracy: 0.8521


---
---